In [1]:
#librerias a usar 
import matplotlib. pyplot as plt
import numpy as np
import random
import scipy
import scipy.special
import math
from scipy.optimize import minimize
import scipy.optimize as op
import scipy.stats as stats
import time
%matplotlib inline

In [2]:
#datos:
tb1=np.loadtxt('jla_mub_covmatrix.dat')
tb2=np.loadtxt('jla_mub.txt')

# divido los datos de jla_mub.txt en dos arrelgos z y mu 
z=tb2[:,0]
mu=tb2[:,1]

In [3]:
#arreglo los datos de jla_mud_covmatrix.dat en una matriz de 31x31
covmatrix=[]
for i in range(31):
    row=[]
    for j in range(31):
        row.append(tb1[(31*i)+j])
    covmatrix.append(row)

In [4]:
#Pongo las varianzas de la matriz anterior (que se encuentran en la diagunal) en un arreglo.  
var=[]
for i in range(31):
    var.append(covmatrix[i][i])

#### Ecuaciones:

\begin{equation}
s^3=\frac{1-\Omega_m}{\Omega_m}
\end{equation}

In [5]:
def sss(omg):
    return ((1.0-omg)/omg)**(1./3)

\begin{equation}
\eta(a,\Omega_m)=2\sqrt{s^3 + 1}\left[ a^{-4} - 0.1540sa^{-3} + 0.4304s^2a^{-2} + 0.19097s^3a^{-1} + 0.066941s^4 \right]^{-1/8}
\end{equation}

In [7]:
def eta(a,omg):
    r = np.sqrt(np.power(sss(omg),3)+1)
    return 2*r*pow((1./a**4 - (0.1540*sss(omg)/(a**3))+(0.4304*pow(sss(omg),2)/(a**2))+(0.19097*pow(sss(omg),3)/(a**1))+(0.066941*pow(sss(omg),4))),-1.0/8)

\begin{equation}
D_L=\frac{c}{H_0} (1+z)\left[ \eta(1,\Omega_m)-\eta\left( \frac{1}{1+z} , \Omega_m \right) \right]
\end{equation}

In [9]:
def D_L(z,omg,H):
    return (299792/H)*(1+z)*(eta(1,omg)-eta((1/(1+z)),omg))

\begin{equation}
\mu_{\lambda CDM} (z;\Omega_m) = 5\log_{10} (d_L(z;\Omega_m)/10pc)
\end{equation}

Con un valor fijo de $H_0 = 70 kms^{-1}Mpc^{-1}$

In [10]:
def mud(z,omg):
    return 25+(5*np.log10(D_L(z,omg,70)))-(5*np.log10(H/100))

La estimación de la distancia con SNe Ia se basa en la observación empírica de que estos eventos forman una clase homogénea cuya variabilidad restante está razonablemente bien captada por dos parámetros. Uno de ellos describe el tiempo de estiramiento de la curvatura de luz $X_1$ y el otro describe el color de supernova a brillo máximo $C$   

Estimador de distancia:

\begin{equation}
\mu = m_B^{\star} - (M_B - \alpha \times X_1 + \beta \times C)
\end{equation}

Donde $m_B^{\star}$ corresponde a la magnitud máxima observada en el marco de reposo B y $\alpha$, $\beta$ y $M_B$ son los nuisance parameters en el estimador de distancia. 


Usando el estimador de distancia anterior, ajustamos una cosmología a las mediciones de supernovas minimizando la siguiente función:


\begin{equation}
\chi^2 = (\hat{\mu} - \mu_{\Lambda CDM}(z;\Omega_m))^{\dagger} C^{-1} (\hat{\mu} - \mu_{\Lambda CDM}(Z;\Omega))
\end{equation}

Donde $C$ es la matriz de covarianza de $\hat{\mu}$. Los parametros libres en el ajuste son $\Omega_m$ y los cuatro nuisance parameters $\alpha, \beta, M_B^{1}$ y $\Delta_M$ 